In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Alex\\Desktop\\R\\machine_learning\\10.Projects\\End-to-end-ML-project-with-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Alex\\Desktop\\R\\machine_learning\\10.Projects\\End-to-end-ML-project-with-MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# Definir una clase de datos inmutable que almacena 
# la configuración para la ingesta de datos
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    # Definir el método constructor que recibe las rutas de los archivos de configuración, parámetros y esquema
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        # Leer los archivos YAML y asignarlos a los atributos de la instancia
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        # Crear el directorio raíz para los artefactos del proyecto
        create_directories([self.config.artifacts_root])


    # Definir un método que devuelve la configuración para la ingesta de datos
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # Obtener la configuración de la ingesta de datos
        config = self.config.data_ingestion 

        # Crear el directorio raíz para la ingesta de datos
        create_directories([config.root_dir])
        # Crear un objeto de la clase DataIngestionConfig con los parámetros de la configuración
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # Definir un método que descarga el archivo de datos de la URL de la fuente
    def download_file(self):
        # Verificar si el archivo local ya existe
        if not os.path.exists(self.config.local_data_file):  
            # Si no existe, descargar el archivo de la URL y guardar el nombre y los encabezados
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            ) 
            # Registrar un mensaje con el nombre del archivo y los encabezados
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        # Asignar la ruta del directorio donde se va a descomprimir el archivo
        unzip_path = self.config.unzip_dir 
        # Crear el directorio si no existe
        os.makedirs(unzip_path, exist_ok=True)
        # Abrir el archivo zip en modo lectura
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            # Extraer todos los archivos del zip al directorio
            zip_ref.extractall(unzip_path)
  

In [12]:
"""
Realizar la ingesta de datos desde una fuente remota 
a un directorio local, usando las clases ConfigurationManager y DataIngestion
"""
try:
    config = ConfigurationManager() # Crear un objeto de la clase ConfigurationManager que gestiona la configuración del proyecto 
    data_ingestion_config = config.get_data_ingestion_config() # Obtener la configuración para la ingesta de datos 
    data_ingestion = DataIngestion(config=data_ingestion_config) # Crear un objeto de la clase DataIngestion que realiza la ingesta de datos usando la configuración obtenida
    data_ingestion.download_file() # Descargar el archivo de datos de la URL
    data_ingestion.extract_zip_file() # Descomprimir el archivo de datos en el directorio local 
except Exception as e:
    raise e

[2023-07-28 17:42:18,840: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-28 17:42:18,846: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-28 17:42:18,852: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-07-28 17:42:18,855: INFO: common: created directory at: artifacts]
[2023-07-28 17:42:18,861: INFO: common: created directory at: artifacts/data_ingestion]


[2023-07-28 17:42:20,447: INFO: 4266687756: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D3A4:45E3:306DC5:396F83:64C4444D
Accept-Ranges: bytes
Date: Fri, 28 Jul 2023 22:42:22 GMT
Via: 1.1 varnish
X-Served-By: cache-lim12123-LIM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1690584142.867261,VS0,VE269
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 44189cb809588d5e49b0ff80a913f92547b8c47b
Expires: Fri, 28 Jul 2023 22:47:22 GMT
Source-Age: 0

]


In [ ]:
# ------------------------------------------------------------------------------------------------